Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/regression/auto-ml-regression.png)

# Automated Machine Learning
_**AutoML Model Testing for Regression**_

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Results Analysis and Test](#Results-Analysis-and-Test)

## Introduction
In this example we use the Hardware Performance dataset to showcase how you can do model training, selection and test with AutoML for forecasting tasks. Besides this example notebook, you can find two other example notebooks with model testing for [classification](../classification/classification-TSI.ipynb) and [forecasting](../forecasting/forecasting-TSI.ipynb). Also, these [docs](https://docs.microsoft.com/en-us/azure/machine-learning/concept-automated-ml#training-validation-and-test-data) and [example code snippets](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-cross-validation-data-splits#provide-test-data-preview) can give you more useful information.

If you are using an Azure Machine Learning Compute Instance, you are all set.  Otherwise, go through the [configuration](../configuration.ipynb)  notebook first if you haven't already to establish your connection to the AzureML Workspace. 

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Configure AutoML using `AutoMLConfig`.
3. Train the model using remote compute.
4. Explore the results.
5. Test the best fitted model and other models.

## Setup

As part of the setup you have already created an Azure ML `Workspace` object. For Automated ML you will need to create an `Experiment` object, which is a named object in a `Workspace` used to run experiments.

In [ ]:
import logging

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

This sample notebook may use features that are not available in previous versions of the Azure ML SDK.

In [ ]:
print("This notebook was created using version 1.40.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

In [ ]:
ws = Workspace.from_config()

# Choose a name for the experiment.
experiment_name = "automl-regression"

experiment = Experiment(ws, experiment_name)

output = {}
output["Subscription ID"] = ws.subscription_id
output["Workspace"] = ws.name
output["Resource Group"] = ws.resource_group
output["Location"] = ws.location
output["Run History Name"] = experiment_name
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
outputDf.T

You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you use `AmlCompute` as your training compute resource.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D2_V2", max_nodes=6
    )
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

## Data

Load the hardware dataset from a csv file containing both training features and labels. The features are inputs to the model, while the training labels represent the expected output of the model. Next, we'll split the data using random_split for a holdout test set.

### Data Splitting

You can use `test_data` parameter in `AutoMLConfig` to pass in the test set, which will be used for getting predictions and metrics for the best model after all model training has completed. When this parameter is specified, a test run will be started after all training runs have completed and the best model has been identified.

To use a train/test split instead of providing test data directly, use the `test_size` parameter when creating the `AutoMLConfig`. In the [example notebook for classification](../classification/classification-TSI.ipynb) you can find an example using `test_size`. This parameter must be a floating point value between 0.0 and 1.0 and specifies the percentage of the training dataset that should be used for the test dataset. For regression based tasks, random sampling is used. For classification tasks, stratified sampling is used. **Forecasting does not currently support specifying a test dataset using a train/test split**.

Note, the `test_data` and `test_size` `AutoMLConfig` parameters are mutually exclusive and can not be specified at the same time.


In [ ]:
data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/machineData.csv"
dataset = Dataset.Tabular.from_delimited_files(data)

# Split the dataset into train and test datasets
train_data, test_data = dataset.random_split(percentage=0.8, seed=223)

label = "ERP"

## Train

Instantiate an `AutoMLConfig` object to specify the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|classification, regression or forecasting|
|**primary_metric**|This is the metric that you want to optimize. Regression supports the following primary metrics: <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**n_cross_validations**|Number of cross validation splits.|
|**training_data**|(sparse) array-like, shape = [n_samples, n_features]|
|**label_column_name**|(sparse) array-like, shape = [n_samples, ], targets values.|

**_You can find more information about primary metrics_** [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train#primary-metric)

In [ ]:
# Use test_data to specify test set for AutoMLRun

automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": "r2_score",
    "enable_early_stopping": True,
    "experiment_timeout_hours": 0.25,  # for real scenarios we reccommend a timeout of at least one hour
    "max_concurrent_iterations": 6,
    "max_cores_per_iteration": -1,
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(
    task="regression",
    compute_target=compute_target,
    training_data=train_data,
    label_column_name=label,
    test_data=test_data,  # Specify test_data
    **automl_settings
)

Call the `submit` method on the experiment object and pass the run configuration. Execution of remote runs is asynchronous. Depending on the data and the number of iterations this can run for a while.  Validation errors and current status will be shown when setting `show_output=True` and the execution will be synchronous.

In [ ]:
remote_run = experiment.submit(automl_config, show_output=False)

In [ ]:
# If you need to retrieve a run that already started, use the following code
# from azureml.train.automl.run import AutoMLRun
# remote_run = AutoMLRun(experiment = experiment, run_id = '<replace with your run id>')

## Results Analysis and Test

### Widget for Monitoring Runs

The widget will first report a "loading" status while running the first iteration. After completing the first iteration, an auto-updating graph and table will be shown. The widget will refresh once per minute, so you should see the graph update as child runs complete.

**Note:** The widget displays a link at the bottom. Use this link to open a web interface to explore the individual run details.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion()

### Retrieve the Best Model

Below we select the best pipeline from our iterations. The `get_output` method returns the best run and the fitted model. The Model includes the pipeline and any pre-processing.  Overloads on `get_output` allow you to retrieve the best run and fitted model for *any* logged metric or for a particular *iteration*.

In [ ]:
best_run, fitted_model = remote_run.get_output()
print(best_run)

In [ ]:
print(fitted_model)

### Get the Predictions and Metrics Generated by the Test Run

Get the test run associated with the best run. When a remote test run is requested by passing in a value for `test_data` or `test_size` to `AutoMLConfig`, only the best training run will have a test run associated with it. To start a test run for models which are not associated with the best run or to start another best run test run, use `ModelProxy`. See the examples later in the notebook for more details.

To see more details about the test run in Azure Machine Learning Studio (view its metrics, get a preview of the predictions, etc...) follow the link to the __Details Page__ listed in the next cells output.

In [ ]:
test_run = next(best_run.get_children(type="automl.model_test"))
test_run.wait_for_completion(show_output=False, wait_post_processing=True)
test_run

Get the __metrics__ from the test run.

In [ ]:
test_run_metrics = test_run.get_metrics()
for name, value in test_run_metrics.items():
    print(f"{name}: {value}")

Get the __predictions__ from the test run.

In [ ]:
test_run_details = test_run.get_details()
test_run_predictions = Dataset.get_by_id(
    ws, test_run_details["outputDatasets"][0]["identifier"]["savedId"]
)
test_run_predictions.to_pandas_dataframe().head()

Start a test run for this new run and get the metrics and predictions.

In [ ]:
from azureml.train.automl.model_proxy import ModelProxy

model_proxy = ModelProxy(best_run)
predictions, test_run_metrics = model_proxy.test(test_data)

predictions.to_pandas_dataframe().head()

In [ ]:
pd.DataFrame.from_dict(test_run_metrics, orient="index", columns=["Value"])

### Model from a Specific Iteration
Show the run and the model from the third iteration:

In [ ]:
iteration = 3
third_run, third_model = remote_run.get_output(iteration=iteration)
print(third_run)

In [ ]:
print(third_model)

Start a test run for this new run and get the metrics and predictions.

In [ ]:
model_proxy = ModelProxy(third_run)
predictions, test_run_metrics = model_proxy.test(test_data)

predictions.to_pandas_dataframe().head()

In [ ]:
pd.DataFrame.from_dict(test_run_metrics, orient="index", columns=["Value"])